In [35]:
class Group:
    def __init__(self, n_units, hit_points, immune_to, weak_to, damage, damage_type, initiative):
        self.n_units = n_units
        self.hit_points = hit_points
        self.damage = damage
        self.damage_type = damage_type
        self.initiative = initiative
        self.immune_to = immune_to
        self.weak_to = weak_to
        self.effective_power = self.n_units * self.damage

    def __repr__(self):
        return f'''
n_units: {self.n_units}
hit_points: {self.hit_points}
damage: {self.damage}
damage_type: {self.damage_type}
initiative: {self.initiative}
immune_to: {', '.join(self.immune_to)}
weak_to: {', '.join(self.weak_to)}
effective_power: {self.effective_power}
'''


In [36]:
immune_system = [
    Group(4082, 2910, [], [], 5, 'cold', 15),
    Group(2820, 9661, ['slashing'], [], 27, 'cold', 8),
    Group(4004, 4885, [], ['slashing'], 10, 'bludgeoning', 13),
    Group(480, 7219, [], ['bludgeoning'], 134, 'radiation', 18),
    Group(8734, 4421, ['bludgeoning'], [], 5, 'slashing', 14),
    Group(516, 2410, [], ['slashing'], 46, 'bludgeoning', 5),
    Group(2437, 11267, [], ['slashing'], 38, 'fire', 17),
    Group(1815, 7239, [], ['cold'], 33, 'slashing', 10),
    Group(4941, 10117, ['bludgeoning'], [], 20, 'fire', 9),
    Group(617, 7816, [], ['bludgeoning', 'slashing'], 120, 'bludgeoning', 4)
]

infection = [
    Group(2877, 20620, [], ['radiation', 'bludgeoning'], 13, 'cold', 11),
    Group(1164, 51797, ['fire'], [], 63, 'fire', 7),
    Group(160, 31039, ['bludgeoning'], ['radiation'], 317, 'bludgeoning', 2),
    Group(779, 24870, ['radiation', 'bludgeoning'], ['slashing'], 59, 'slashing', 12),
    Group(1461, 28000, ['radiation'], ['bludgeoning'], 37, 'slashing', 16),
    Group(1060, 48827, [], [], 73, 'slashing', 3),
    Group(4422, 38291, [], [], 14, 'slashing', 1),
    Group(4111, 14339, ['fire', 'bludgeoning', 'cold'], [], 6, 'radiation', 20),
    Group(4040, 49799, ['bludgeoning', 'cold'], ['slashing', 'fire'], 24, 'fire', 19),
    Group(2198, 41195, [], ['radiation'], 36, 'slashing', 6)

]

In [37]:
for g in immune_system:
    print(g)


n_units: 4082
hit_points: 2910
damage: 5
damage_type: cold
initiative: 15
immune_to: 
weak_to: 
effective_power: 20410


n_units: 2820
hit_points: 9661
damage: 27
damage_type: cold
initiative: 8
immune_to: slashing
weak_to: 
effective_power: 76140


n_units: 4004
hit_points: 4885
damage: 10
damage_type: bludgeoning
initiative: 13
immune_to: 
weak_to: slashing
effective_power: 40040


n_units: 480
hit_points: 7219
damage: 134
damage_type: radiation
initiative: 18
immune_to: 
weak_to: bludgeoning
effective_power: 64320


n_units: 8734
hit_points: 4421
damage: 5
damage_type: slashing
initiative: 14
immune_to: bludgeoning
weak_to: 
effective_power: 43670


n_units: 516
hit_points: 2410
damage: 46
damage_type: bludgeoning
initiative: 5
immune_to: 
weak_to: slashing
effective_power: 23736


n_units: 2437
hit_points: 11267
damage: 38
damage_type: fire
initiative: 17
immune_to: 
weak_to: slashing
effective_power: 92606


n_units: 1815
hit_points: 7239
damage: 33
damage_type: slashing
initiati